In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import librosa
# import tensorflow as tf
import glob
import numpy as np

In [3]:
# follow hyperparameters from here, https://github.com/pytorch/fairseq/tree/master/examples/wav2vec

features = [(512, 10, 5), (512, 8, 4), (512, 4, 2), (512, 4, 2), (512, 4, 2), (512, 1, 1), (512, 1, 1)]
aggs = [(512, 2, 1), (512, 3, 1), (512, 4, 1), (512, 5, 1), (512, 6, 1), (512, 7, 1), (512, 8, 1), (512, 9, 1), 
        (512, 10, 1), (512, 11, 1), (512, 12, 1), (512, 13, 1)]
num_negatives = 10
prediction_steps = 12
learning_rate = 1e-5
min_learning_rate = 1e-9
max_learning_rate = 0.005
learning_scheduler = 'cosine'
max_update = 400000
residual_scale = 0.5
log_compression = True
warmup_updates = 50
warmup_init_lr = 1e-07
batch_size = 32
epoch = 10
total_steps = batch_size * epoch

In [4]:
import torch
from torch import nn
import torch.functional as F

In [5]:
np.random.seed(1)
x = np.random.normal(size = (2, 10, 7))
x

array([[[ 1.62434536, -0.61175641, -0.52817175, -1.07296862,
          0.86540763, -2.3015387 ,  1.74481176],
        [-0.7612069 ,  0.3190391 , -0.24937038,  1.46210794,
         -2.06014071, -0.3224172 , -0.38405435],
        [ 1.13376944, -1.09989127, -0.17242821, -0.87785842,
          0.04221375,  0.58281521, -1.10061918],
        [ 1.14472371,  0.90159072,  0.50249434,  0.90085595,
         -0.68372786, -0.12289023, -0.93576943],
        [-0.26788808,  0.53035547, -0.69166075, -0.39675353,
         -0.6871727 , -0.84520564, -0.67124613],
        [-0.0126646 , -1.11731035,  0.2344157 ,  1.65980218,
          0.74204416, -0.19183555, -0.88762896],
        [-0.74715829,  1.6924546 ,  0.05080775, -0.63699565,
          0.19091548,  2.10025514,  0.12015895],
        [ 0.61720311,  0.30017032, -0.35224985, -1.1425182 ,
         -0.34934272, -0.20889423,  0.58662319],
        [ 0.83898341,  0.93110208,  0.28558733,  0.88514116,
         -0.75439794,  1.25286816,  0.51292982],
        [-

In [6]:
x = torch.from_numpy(x)
x.shape

torch.Size([2, 10, 7])

In [7]:
bsz, fsz, tsz = x.shape
bsz, fsz, tsz

(2, 10, 7)

In [8]:
y = x.transpose(0, 1)
y = y.contiguous().view(fsz, -1)
y.shape

torch.Size([10, 14])

In [9]:
high = tsz
n_negatives = 10
# neg_idxs = torch.randint(low=0, high=high, size=(bsz, n_negatives * tsz))
# neg_idxs

In [10]:
neg_idxs = torch.from_numpy(np.array([[
         1, 2, 3, 1, 4, 0, 5, 6, 1, 2, 0, 4, 2, 1, 0, 5, 4, 5, 4, 6, 6, 4, 1, 6,
         6, 3, 4, 4, 5, 0, 1, 5, 4, 4, 1, 1, 0, 2, 0, 6, 2, 6, 3, 4, 5, 6, 2, 4,
         0, 2, 1, 2, 6, 4, 2, 4, 0, 2, 4, 2, 1, 0, 4, 6, 6, 4, 4, 2, 3, 4],
        [4, 0, 3, 4, 2, 4, 4, 1, 0, 6, 3, 1, 5, 6, 4, 3, 6, 4, 0, 5, 1, 0, 4, 2,
         2, 0, 4, 1, 4, 3, 2, 2, 0, 4, 2, 3, 4, 6, 6, 2, 4, 0, 3, 1, 6, 2, 4, 5,
         1, 3, 1, 3, 3, 1, 3, 0, 3, 6, 0, 5, 2, 4, 5, 6, 0, 1, 2, 3, 6, 3]]))

In [11]:
for i in range(1, bsz):
    neg_idxs[i] += i * high
    
neg_idxs

tensor([[ 1,  2,  3,  1,  4,  0,  5,  6,  1,  2,  0,  4,  2,  1,  0,  5,  4,  5,
          4,  6,  6,  4,  1,  6,  6,  3,  4,  4,  5,  0,  1,  5,  4,  4,  1,  1,
          0,  2,  0,  6,  2,  6,  3,  4,  5,  6,  2,  4,  0,  2,  1,  2,  6,  4,
          2,  4,  0,  2,  4,  2,  1,  0,  4,  6,  6,  4,  4,  2,  3,  4],
        [11,  7, 10, 11,  9, 11, 11,  8,  7, 13, 10,  8, 12, 13, 11, 10, 13, 11,
          7, 12,  8,  7, 11,  9,  9,  7, 11,  8, 11, 10,  9,  9,  7, 11,  9, 10,
         11, 13, 13,  9, 11,  7, 10,  8, 13,  9, 11, 12,  8, 10,  8, 10, 10,  8,
         10,  7, 10, 13,  7, 12,  9, 11, 12, 13,  7,  8,  9, 10, 13, 10]])

In [12]:
negs = y[..., neg_idxs.view(-1)]
negs.shape

torch.Size([10, 140])

In [13]:
negs

tensor([[-0.6118, -0.5282, -1.0730,  ...,  0.8762, -0.3062,  0.8762],
        [ 0.3190, -0.2494,  1.4621,  ..., -0.2223,  0.4101, -0.2223],
        [-1.0999, -0.1724, -0.8779,  ...,  0.3776,  1.1989,  0.3776],
        ...,
        [ 0.3002, -0.3522, -1.1425,  ...,  1.2738,  1.2362,  1.2738],
        [ 0.9311,  0.2856,  0.8851,  ...,  0.8633, -1.2301,  0.8633],
        [ 0.4885, -0.0756,  1.1316,  ...,  0.5206,  0.0466,  0.5206]],
       dtype=torch.float64)

In [14]:
negs = negs.view(fsz, bsz, n_negatives, tsz).permute(2, 1, 0, 3)
negs.shape

torch.Size([10, 2, 10, 7])

In [15]:
negs[0]

tensor([[[-0.6118, -0.5282, -1.0730, -0.6118,  0.8654,  1.6243, -2.3015],
         [ 0.3190, -0.2494,  1.4621,  0.3190, -2.0601, -0.7612, -0.3224],
         [-1.0999, -0.1724, -0.8779, -1.0999,  0.0422,  1.1338,  0.5828],
         [ 0.9016,  0.5025,  0.9009,  0.9016, -0.6837,  1.1447, -0.1229],
         [ 0.5304, -0.6917, -0.3968,  0.5304, -0.6872, -0.2679, -0.8452],
         [-1.1173,  0.2344,  1.6598, -1.1173,  0.7420, -0.0127, -0.1918],
         [ 1.6925,  0.0508, -0.6370,  1.6925,  0.1909, -0.7472,  2.1003],
         [ 0.3002, -0.3522, -1.1425,  0.3002, -0.3493,  0.6172, -0.2089],
         [ 0.9311,  0.2856,  0.8851,  0.9311, -0.7544,  0.8390,  1.2529],
         [ 0.4885, -0.0756,  1.1316,  0.4885,  1.5198, -0.2981,  2.1856]],

        [[ 0.3156, -1.4441,  0.8762,  0.3156,  0.1600,  0.3156,  0.3156],
         [-0.2008,  0.8280, -0.2223, -0.2008,  0.7620, -0.2008, -0.2008],
         [ 0.1218,  0.1983,  0.3776,  0.1218, -0.6707,  0.1218,  0.1218],
         [ 0.0773,  0.1852,  0.4235,

In [16]:
y = x[:].unsqueeze(0)
print(y.shape, negs.shape)
targets = torch.cat([y, negs], dim=0)
targets.shape

torch.Size([1, 2, 10, 7]) torch.Size([10, 2, 10, 7])


torch.Size([11, 2, 10, 7])

In [17]:
project_to_steps = nn.ConvTranspose2d(10, 10, (1, 12))
s = project_to_steps(x.unsqueeze(-1).float()).unsqueeze(0).expand(targets.size(0), -1, -1, -1, -1)
s.shape

torch.Size([11, 2, 10, 7, 12])

In [18]:
import pickle
with open('convtranspose.pkl', 'wb') as fopen:
    pickle.dump(s.detach().numpy().tolist(), fopen)

In [19]:
import math

jin = 0
rin = 0
for _, k, stride in features:
    if rin == 0:
        rin = k
    rin = rin + (k - 1) * jin
    if jin == 0:
        jin = stride
    else:
        jin *= stride
offset = math.ceil(rin / jin)

offset = int(offset)
print(offset)

3


In [20]:
copies, bsz, dim, tsz, steps = s.shape
steps = min(steps, tsz - offset)
predictions = s.new(bsz * copies * (tsz - offset + 1) * steps - ((steps + 1) * steps // 2) * copies * bsz)
labels = torch.zeros_like(predictions)
predictions.shape, labels.shape

(torch.Size([220]), torch.Size([220]))

In [21]:
s.shape, targets.shape, s[..., :-offset, i].shape, targets[..., offset:].shape

(torch.Size([11, 2, 10, 7, 12]),
 torch.Size([11, 2, 10, 7]),
 torch.Size([11, 2, 10, 4]),
 torch.Size([11, 2, 10, 4]))

In [22]:
start = end = 0
for i in range(steps):
    offset = i + offset
    end = start + (tsz - offset) * bsz * copies
    pos_num = (end - start) // copies
    print(start, pos_num, end, offset, s[..., :-offset, i].shape, targets[..., offset:].shape)
    predictions[start:end] = (s[..., :-offset, i].float() * targets[..., offset:].float()).sum(dim=2).flatten()
    print(labels[start:start + pos_num])
    labels[start:start + pos_num] = 1.
    start = end
    
labels

0 8 88 3 torch.Size([11, 2, 10, 4]) torch.Size([11, 2, 10, 4])
tensor([0., 0., 0., 0., 0., 0., 0., 0.])
88 6 154 4 torch.Size([11, 2, 10, 3]) torch.Size([11, 2, 10, 3])
tensor([0., 0., 0., 0., 0., 0.])
154 2 176 6 torch.Size([11, 2, 10, 1]) torch.Size([11, 2, 10, 1])
tensor([0., 0.])
176 -4 132 9 torch.Size([11, 2, 10, 0]) torch.Size([11, 2, 10, 0])
tensor([])


tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [30]:
predictions.detach().numpy()[:-4 * 11]

array([ 3.66452038e-01,  3.81497800e-01,  1.17975384e-01,  3.86789769e-01,
       -5.22472978e-01,  2.96876747e-02,  1.16604976e-02, -2.87122130e-01,
       -5.71662426e-01,  3.81497800e-01,  1.39530897e-01,  4.34068859e-01,
       -6.46445900e-04,  5.10506220e-02,  1.72045007e-01, -1.56508684e-02,
        3.22836012e-01,  3.81497800e-01,  1.98190331e-01,  6.12615168e-01,
       -5.22472978e-01,  7.26154149e-02,  1.16604976e-02, -2.87122130e-01,
        1.93178654e-04,  3.81497800e-01,  3.89128476e-01,  3.86789769e-01,
       -6.46445900e-04, -1.97719205e-02,  1.16604976e-02,  1.03846192e-01,
       -3.02614093e-01, -7.46433139e-01,  4.23885345e-01,  6.11973643e-01,
        4.69868928e-01, -1.97719205e-02,  1.72045007e-01,  1.03846192e-01,
        1.93178654e-04,  3.81497800e-01,  4.23885345e-01,  6.12615168e-01,
        4.69868928e-01, -1.97719205e-02,  1.72045007e-01, -1.56631157e-01,
        3.22836012e-01,  1.09955943e+00,  1.98190331e-01,  3.86789769e-01,
        2.98553944e-01,  

In [24]:
# torch.nn.functional.binary_cross_entropy(predictions, labels)

In [25]:
torch.nn.functional.sigmoid(predictions)

/home/husein/.local/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([0.5906, 0.5942, 0.5295, 0.5955, 0.3723, 0.5074, 0.5029, 0.4287, 0.3609,
        0.5942, 0.5348, 0.6068, 0.4998, 0.5128, 0.5429, 0.4961, 0.5800, 0.5942,
        0.5494, 0.6485, 0.3723, 0.5181, 0.5029, 0.4287, 0.5000, 0.5942, 0.5961,
        0.5955, 0.4998, 0.4951, 0.5029, 0.5259, 0.4249, 0.3216, 0.6044, 0.6484,
        0.6154, 0.4951, 0.5429, 0.5259, 0.5000, 0.5942, 0.6044, 0.6485, 0.6154,
        0.4951, 0.5429, 0.4609, 0.5800, 0.7502, 0.5494, 0.5955, 0.5741, 0.5128,
        0.5429, 0.4211, 0.4249, 0.5117, 0.6044, 0.3740, 0.6154, 0.5074, 0.5029,
        0.5259, 0.4249, 0.5942, 0.5494, 0.6484, 0.3723, 0.5181, 0.5333, 0.4211,
        0.5807, 0.5257, 0.5348, 0.6484, 0.1694, 0.5128, 0.5429, 0.6010, 0.5751,
        0.5117, 0.5027, 0.6484, 0.6154, 0.4088, 0.5744, 0.4512, 0.3980, 0.4677,
        0.5400, 0.4127, 0.4610, 0.5907, 0.3980, 0.4326, 0.4598, 0.4167, 0.4440,
        0.5640, 0.3980, 0.4577, 0.3910, 0.6306, 0.4610, 0.5907, 0.3980, 0.4534,
        0.5400, 0.4899, 0.4610, 0.4235, 